In [192]:
import string

# settings
window_size = 3
text_file = "C:\\Users\\sures\\Downloads\\coco_val_sentences.txt"


# functions
def preprocessor_word(word):
    word = word.lower().strip()
    for punc in string.punctuation:
        word = word.replace(punc,"")
    return(word)

# main

# Initialize
word_training_pairs = [] # (target, context) pairs

# load text
all_sentences = open(text_file).readlines()

# extract context

for sentence in all_sentences:
    sentence_splitted = [preprocessor_word(word) for word in sentence.split()]
    for i, target in enumerate(sentence_splitted):
        for j in range(1,4):
            if not i + j >= len(sentence_splitted):
                word_training_pairs.append((target, sentence_splitted[i+j]))
            if not i - j < 0:
                word_training_pairs.append((target, sentence_splitted[i-j]))


#print(word_training_pairs)

id2word = list(set([pair[0] for pair in word_training_pairs]))
#print(id2word)
word2id = {w:i for i, w in enumerate (id2word)}
word_training_pairs = [(word2id[pair[0]], word2id[pair[1]]) for pair in word_training_pairs  ]

In [193]:
len(word2id)

18309

In [194]:
import tensorflow as tf
import numpy as np


In [195]:
tf.reset_default_graph
vocabulary_size = len(id2word)
embedding_size = 50
batch_size = 100


# variables

w_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size]))
softmax_weights =  tf.Variable(tf.random_uniform([embedding_size,vocabulary_size]))

# input

train_pairs = tf.placeholder(tf.int32, shape = [None,2])
train_input = train_pairs[:,0]
train_output = train_pairs[:,1]

# model

w_embd = tf.nn.embedding_lookup(w_embeddings,train_input)
prediction = tf.matmul(w_embd, softmax_weights)

# loss

loss = tf.losses.sparse_softmax_cross_entropy(train_output, prediction)

# optimizer

optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

init = tf.global_variables_initializer()

loss_hist =[]
with tf.Session() as sess:
    sess.run(init)
    for i in range(0, len(word_training_pairs), batch_size):
                batch_data = word_training_pairs[i:i+batch_size]
                #print(len(batch_data))
                _, loss_value = sess.run([optimizer, loss], feed_dict = {train_pairs:batch_data})
                loss_hist.append(loss_hist)
                   
    W = sess.run(w_embeddings)
        

In [202]:
W.shape

(18309, 50)

In [201]:
W

array([[-0.11776736, -0.02214624,  0.25439596, ...,  0.17768185,
        -0.94560534,  0.7690013 ],
       [ 0.18271548,  0.15488063,  0.19089527, ..., -0.19574301,
         0.5605203 ,  1.0901375 ],
       [ 0.20374803, -0.03974609, -0.45830786, ...,  0.7548938 ,
         0.47588712, -0.02808652],
       ...,
       [-0.26966155,  0.64500356,  1.0570655 , ...,  0.5889324 ,
        -0.30192262,  0.28903794],
       [-0.28609926, -0.17132397, -0.10276785, ...,  0.6116154 ,
         0.29895425,  0.29451463],
       [ 0.31538153,  0.9849304 ,  1.8993207 , ...,  1.1043595 ,
         0.13660835,  0.9297342 ]], dtype=float32)

In [205]:
np.save("W_coco_val_newer.npy", W)
#W = np.load("W.npy")

In [206]:
f = open("coco_val_newer.txt","w")
for w in id2word:
    f.write(w+"\n")
f.close()

In [203]:
f

<_io.TextIOWrapper name='coco_val_new.txt' mode='w' encoding='cp1252'>